In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
globalAA = pd.read_csv('queries_results/clustering/A.csv', header = 0, sep=' *, *')
# globalA["Time"]=pd.to_datetime(globalA['MonthYear'], format='%Y%m').dt.strftime('%Y-%m')
globalA = globalAA[['Events','sumNumMentions','materialConfCoop','verbalConfCoop','avgAvgTone','avgGoldstein','fightCount','expressCount']]

/home/micha/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
points = globalA
points = points.to_numpy()
points

array([[ 7.03800000e+03,  3.46670000e+04,  7.01903950e-02, ...,
         1.47698210e+00,  1.40000000e+02,  7.70000000e+01],
       [ 6.28210000e+04,  3.07088000e+05,  1.21376610e-01, ...,
         1.24050875e+00,  3.45000000e+03,  1.13400000e+03],
       [ 5.31000000e+02,  2.71000000e+03,  4.33145009e-02, ...,
         2.16610169e+00,  1.00000000e+01,  1.00000000e+01],
       ...,
       [ 8.19808000e+05,  3.47227000e+06,  1.57140452e-01, ...,
         3.70861958e-01,  5.77160000e+04,  1.97880000e+04],
       [ 1.88650000e+04,  9.09150000e+04,  2.24118738e-01, ...,
        -3.64373178e-01,  2.95600000e+03,  2.58000000e+02],
       [ 2.61000000e+02,  1.42000000e+03,  5.36398467e-02, ...,
         2.74904215e+00,  2.00000000e+00,  4.00000000e+00]])

### Data standardisation with [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [4]:
##jeśli dane są w jakiś sposób skorelowane to znaczy, że występuje redundancja danych, te dane nic nowego nie wnoszą
##są niepotrzebne

## if necessary to scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_points = scaler.fit_transform(points)

## Find clusters using [kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [5]:
#klasa KMeans z sklearn, konstruktor przyjmuje liczbę klastrów i sposób inicjalizacji centroidów, random_state=1 za
#każdym razem te same punkty startowe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='random', random_state=1)
km = kmeans.fit(scaled_points)

In [6]:
#funkcja predict zwaraca etykiety klastru do jakiego został przypisany obiekt
clusters_id = kmeans.predict(scaled_points)
clusters_id

array([5, 6, 7, 6, 6, 5, 0, 6, 1, 4, 4, 4, 5, 8, 8, 6, 0, 6, 5, 6, 1, 6,
       0, 6, 4, 5, 6, 4, 6, 4, 6, 1, 5, 6, 0, 4, 3, 5, 0, 6, 4, 5, 0, 5,
       4, 0, 5, 5, 6, 5, 0, 0, 0, 2, 8, 5, 6, 6, 5, 4, 2, 5, 6, 7, 4, 1,
       6, 0, 6, 5, 2, 0, 4, 0, 4, 0, 6, 6, 0, 6, 4, 4, 6, 6, 0, 4, 7, 5,
       6, 4, 6, 6, 1, 6, 5, 6, 6, 6, 6, 2, 0, 6, 0, 4, 0, 5, 4, 0, 5, 8,
       6, 0, 6, 5, 6, 1, 5, 5, 0, 6, 5, 6, 1, 0, 5, 5, 1, 5, 4, 5, 4, 4,
       2, 6, 4, 6, 6, 2, 5, 4, 5, 5, 6, 4, 4, 5, 7, 5, 6, 5, 1, 5, 1, 4,
       6, 6, 5, 5, 4, 0, 1, 6, 6, 7, 6, 1, 5, 5, 5, 5, 6, 4, 6, 5, 6, 0,
       5, 6, 0, 4, 6, 5, 5, 4, 7, 0, 1, 4, 7, 8, 2, 4, 8, 5, 4, 5, 0, 5,
       6, 0, 0, 1, 5, 4, 1, 6, 1, 6, 6, 5, 7, 6, 6, 0, 4, 5, 9, 1, 4],
      dtype=int32)

In [7]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = globalAA['Actor1CountryCode']
cluster_map['cluster'] = km.labels_

In [8]:
cluster_map


,data_index,cluster
0,DZA,5
1,TUR,6
2,SYC,7
3,ZWE,6
4,MWI,6
...,...,...
214,CRI,4
215,GNB,5
216,USA,9
217,AFG,1


In [9]:
POL = cluster_map[cluster_map.data_index=='POL'].cluster.iloc[0]
cluster_map[cluster_map.cluster==POL]

,data_index,cluster
1,TUR,6
3,ZWE,6
4,MWI,6
7,MCO,6
15,RUS,6
17,BDI,6
19,JPN,6
21,NLD,6
23,CAN,6
26,ALB,6


In [10]:
USA = cluster_map[cluster_map.data_index=='USA'].cluster.iloc[0]
cluster_map[cluster_map.cluster==USA]

,data_index,cluster
216,USA,9


In [11]:
DEU = cluster_map[cluster_map.data_index=='DEU'].cluster.iloc[0]
cluster_map[cluster_map.cluster==DEU]

,data_index,cluster
1,TUR,6
3,ZWE,6
4,MWI,6
7,MCO,6
15,RUS,6
17,BDI,6
19,JPN,6
21,NLD,6
23,CAN,6
26,ALB,6


In [12]:
RUS = cluster_map[cluster_map.data_index=='RUS'].cluster.iloc[0]
cluster_map[cluster_map.cluster==RUS]

,data_index,cluster
1,TUR,6
3,ZWE,6
4,MWI,6
7,MCO,6
15,RUS,6
17,BDI,6
19,JPN,6
21,NLD,6
23,CAN,6
26,ALB,6


In [13]:
UKR = cluster_map[cluster_map.data_index=='UKR'].cluster.iloc[0]
cluster_map[cluster_map.cluster==UKR]

,data_index,cluster
8,SLV,1
20,PHL,1
31,CMR,1
65,TJK,1
92,MLI,1
115,NER,1
122,BGD,1
126,LBN,1
150,SVK,1
152,MDV,1


In [14]:
GBR = cluster_map[cluster_map.data_index=='GBR'].cluster.iloc[0]
cluster_map[cluster_map.cluster==GBR]

,data_index,cluster
1,TUR,6
3,ZWE,6
4,MWI,6
7,MCO,6
15,RUS,6
17,BDI,6
19,JPN,6
21,NLD,6
23,CAN,6
26,ALB,6


In [15]:
CHN = cluster_map[cluster_map.data_index=='CHN'].cluster.iloc[0]
cluster_map[cluster_map.cluster==CHN]

,data_index,cluster
6,PRY,0
16,JAM,0
22,HND,0
34,VNM,0
38,COG,0
42,MNG,0
45,SSD,0
50,SHN,0
51,SDN,0
52,CHN,0
